In [324]:
import pandas as pd
import numpy as np
import re
import sagemaker
import boto3

In [325]:
from sagemaker import get_execution_role
role = get_execution_role()

In [326]:
s3 = boto3.client('s3')

In [327]:
user_table_file = s3.get_object(Bucket='mymacrochefhj', Key='Resources/Data/kmeans_output_users_cluster.csv')
user_table = pd.read_csv(user_table_file['Body'])

In [328]:
if 'cluster' in list(user_table.columns):
    user_table.drop('cluster',axis=1,inplace=True)

In [329]:
from sagemaker import KMeans

In [330]:
num_clusters = 30
kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.c4.xlarge',
                output_path='s3://sagemakeroutputhj/',              
                k=num_clusters)

In [331]:
train_data = np.array(user_table.drop('user_id',axis=1)).astype('float32')

In [ ]:
%%time
kmeans.fit(kmeans.record_set(train_data))

2019-12-21 03:51:07 Starting - Starting the training job..

In [ ]:
%%time
kmeans_predictor = kmeans.deploy(initial_instance_count=1, 
                                 instance_type='ml.t2.medium')

In [71]:
result=kmeans_predictor.predict(train_data)

In [72]:
res1 = []
for i in range(len(result)):
    res1.append(result[i].label['closest_cluster'].float32_tensor.values[0])

In [74]:
user_table['cluster'] = res1

In [75]:
user_table.to_csv('kmeans_output_users_cluster.csv', header=True, index=False)

In [ ]:
user_table.loc[0].values

In [70]:
kmeans_predictor.endpoint

'kmeans-2019-12-17-23-25-25-039'

sagemaker.Session().delete_endpoint(kmeans_predictor.endpoint)

In [291]:
result = {'predictions': [{'distance_to_cluster': 31.28098487854004, 'closest_cluster': 4.0}]}
clusterfile = s3.get_object(Bucket='mymacrochefhj', Key='Resources/Data/kmeans_output_users_cluster.csv')
clusterDF = pd.read_csv(clusterfile['Body'])
predicted_cluster = result['predictions'][0]['closest_cluster']
newclusterDF = clusterDF[clusterDF['cluster'] == predicted_cluster]
# print (clusterDF[clusterDF['cluster'] == predicted_cluster]['user_id'].head(10))
mealsfile = s3.get_object(Bucket='mymacrochefhj', Key='Resources/Data/final_meals.csv')
mealsDF = pd.read_csv(mealsfile['Body'])
ratingfile = s3.get_object(Bucket='mymacrochefhj', Key='Resources/Data/meals_data.csv')
ratingDF = pd.read_csv(ratingfile['Body'])
print(ratingDF.head(3))

   user_id  meal_id  ratings  meal_type
0        0     3793        6  breakfast
1        5     3793        4  breakfast
2        8     3793        8  breakfast


In [256]:
a = 'vegan, ovovegan, lacvegan, lacovovegan, pesce'

In [257]:
a.split(', ')

['vegan', 'ovovegan', 'lacvegan', 'lacovovegan', 'pesce']

In [258]:
mealsDF.columns = ['meal_id', 'name', 'meal_type', 'vegan', 'ovovegan', 'lacvegan', 'lacovovegan', 'pesce']

In [198]:
shellfish=['shrimp','prawn','crawfish','lobster','squid','scallop']
soy=['soybean','soy milk','soy sauce']
milk=['milk','cheese','butter','margarine','yogurt','cream']
egg=["egg"]
peanut=['peanut']
treenut=['nut','almond','cashew','pistachio']
wheat=['dough','crust','bread','flour']
fish=['fish']
diabetic=['sugar','chocolate','ice cream']
gluten=['wheat','barley','rye','oats','dough','crust','bread']
glutenfree=['rice flour']
seafood=['swordfish','tuna','shark','whale','cod','dolphin','salmon','sardine','catfish','eel','anchovy','shrimp','crab','lobster','krill','jellyfish','octopus','clam','oyster','squid','seaweed','prawn','crawfish','scallop']
redmeat=['horse','pork','beef','bison','lamb','bacon','ham','sausage','buffalo','ostrich','venison','salami','mutton','goat']
whitemeat=['chicken','duck','turkey','goose','pigeon','quail','rabbit','squab','snake','veal']

In [199]:
food_data['shellfish'] = food_data.ingredients.str.contains('|'.join(shellfish), case=False) * 1
food_data['soy'] = food_data.ingredients.str.contains('|'.join(soy), case=False) * 1
food_data['milk'] = food_data.ingredients.str.contains('|'.join(milk), case=False) * 1
food_data['egg'] = food_data.ingredients.str.contains('|'.join(egg), case=False) * 1
food_data['peanut'] = food_data.ingredients.str.contains('|'.join(peanut), case=False) * 1
food_data['treenut'] = food_data.ingredients.str.contains('|'.join(treenut), case=False) * 1
food_data['wheat'] = food_data.ingredients.str.contains('|'.join(wheat), case=False) * 1
food_data['fish'] = food_data.ingredients.str.contains('|'.join(fish), case=False) * 1
food_data['diabetic'] = food_data.ingredients.str.contains('|'.join(diabetic), case=False) * 1
food_data['gluten'] = food_data.ingredients.str.contains('|'.join(gluten), case=False) * 1
food_data['glutenfree'] = food_data.ingredients.str.contains('|'.join(glutenfree), case=False) * 1
food_data['seafood'] = food_data.ingredients.str.contains('|'.join(seafood), case=False) * 1
food_data['redmeat'] = food_data.ingredients.str.contains('|'.join(redmeat), case=False) * 1
food_data['whitemeat'] = food_data.ingredients.str.contains('|'.join(whitemeat), case=False) * 1

In [201]:
mealsDF = food_data.copy()

In [202]:
mealsDF[mealsDF['meal_type'] == 'snack'].shape

(4431, 20)

In [ ]:
vegan =  no milk and nonveg , no egg row['egg'] or row['fish'] or row['seafood'] or row['redmeat'] or row['whitemeat']
lac veg = milk + vegan
ovo veg = egg +  vegan
ovo lac veg = egg + milk + vegan
pescetarian = fish + seafood + vegan

In [205]:
def vegan(row):
    if row['egg'] or row['fish'] or row['seafood'] or row['redmeat'] or row['whitemeat'] or row['milk'] or row['shellfish']:
        return 0
    return 1

In [206]:
mealsDF['vegan'] = mealsDF.apply(vegan, axis=1)

In [207]:
def lac_vegan(row):
    if row['vegan'] or row['milk'] and not (row['egg'] or row['fish'] or row['seafood'] or row['redmeat'] or row['whitemeat'] or row['shellfish']):
        return 1
    return 0

In [208]:
mealsDF['lac_vegan'] = mealsDF.apply(lac_vegan, axis=1)

In [209]:
def ovo_vegan(row):
    if row['vegan'] or row['egg'] and not (row['milk'] or row['fish'] or row['seafood'] or row['redmeat'] or row['whitemeat'] or row['shellfish']):
        return 1
    return 0

In [210]:
mealsDF['ovo_vegan'] = mealsDF.apply(ovo_vegan, axis=1)

In [211]:
def ovo_lac_vegan(row):
    if row['vegan'] or row['milk'] or row['egg'] and not (row['fish'] or row['seafood'] or row['redmeat'] or row['whitemeat'] or row['shellfish']):
        return 1
    return 0

In [212]:
mealsDF['ovo_lac_vegan'] = mealsDF.apply(ovo_lac_vegan, axis=1)

In [213]:
def pescetarian(row):
    if row['vegan'] or row['fish'] or row['seafood'] and not (row['egg'] or row['milk'] or row['redmeat'] or row['whitemeat'] or row['shellfish']):
        return 1
    return 0

In [214]:
mealsDF['pescetarian'] = mealsDF.apply(pescetarian, axis=1)

In [266]:
tempDF.columns

Index(['meal_id', 'name', 'meal_type', 'vegan', 'lac_vegan', 'ovo_vegan',
       'ovo_lac_vegan', 'pescetarian'],
      dtype='object')

In [292]:
mealsDF.columns

Index(['meal_id', 'name', 'meal_type', 'vegan', 'ovovegan', 'lacvegan',
       'lacovovegan', 'pesce'],
      dtype='object')

In [218]:
from io import StringIO

In [261]:
csv_buffer = StringIO()
mealsDF.to_csv(csv_buffer, index=False)

In [263]:
t = s3.put_object(Bucket='mymacrochefhj', Key='Resources/Data/final_meals.csv', Body=csv_buffer.getvalue())

In [295]:
temp = s3.get_object(Bucket='mymacrochefhj', Key='Resources/Data/final_meals.csv')
temp = pd.read_csv(temp['Body'])

In [297]:
for d in ['vegan', 'ovovegan', 'lacvegan', 'lacovovegan', 'pesce']:
    selected_meals = pd.merge(newclusterDF, ratingDF, left_on='user_id', right_on='user_id', how='inner')
    selected_meals = selected_meals[['user_id','meal_id','ratings']]
    selected_meals = pd.merge(selected_meals, temp, left_on='meal_id', right_on='meal_id', how='inner')
    print(selected_meals.columns)
    selected_meals = selected_meals[selected_meals[d] == 1]
    print(selected_meals.shape)
    breakfast_meals = selected_meals[selected_meals.meal_type == 'breakfast'].sort_values(by='ratings', ascending=False)

    breakfast_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(breakfast_meals.shape)

    b = list(breakfast_meals.head(28)['meal_id'])
    bf_meals = []
    for i in b:
        bf_meals.append(mealsDF[mealsDF['meal_id']==i]['name'].values[0])
    lunch_meals = selected_meals[selected_meals.meal_type == 'lunch'].sort_values(by='ratings', ascending=False)

    lunch_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(lunch_meals.shape)

    l = list(lunch_meals.head(28)['meal_id'])
    l_meals = []
    for i in l:
        l_meals.append(mealsDF[mealsDF['meal_id']==i]['name'].values[0])

    dinner_meals = selected_meals[selected_meals.meal_type == 'dinner'].sort_values(by='ratings', ascending=False)

    dinner_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(dinner_meals.shape)

    d = list(dinner_meals.head(28)['meal_id'])
    d_meals = []
    for i in d:
        d_meals.append(mealsDF[mealsDF['meal_id']==i]['name'].values[0]) 

    snack_meals = selected_meals[selected_meals.meal_type == 'snack'].sort_values(by='ratings', ascending=False)

    snack_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(snack_meals.shape)

    s = list(snack_meals.head(28)['meal_id']) 
    s_meals = []
    for i in s:
        s_meals.append(mealsDF[mealsDF['meal_id']==i]['name'].values[0])
    
#     print(s_meals)
#     print(bf_meals)
#     print(l_meals)
#     print(d_meals)
    
    print('\n\n\n')

Index(['user_id', 'meal_id', 'ratings', 'name', 'meal_type', 'vegan',
       'ovovegan', 'lacvegan', 'lacovovegan', 'pesce'],
      dtype='object')
(1387, 10)
(17, 10)
(12, 10)
(7, 10)
(15, 10)




Index(['user_id', 'meal_id', 'ratings', 'name', 'meal_type', 'vegan',
       'ovovegan', 'lacvegan', 'lacovovegan', 'pesce'],
      dtype='object')
(2510, 10)
(27, 10)
(22, 10)
(12, 10)
(33, 10)




Index(['user_id', 'meal_id', 'ratings', 'name', 'meal_type', 'vegan',
       'ovovegan', 'lacvegan', 'lacovovegan', 'pesce'],
      dtype='object')
(1776, 10)
(21, 10)
(16, 10)
(9, 10)
(19, 10)




Index(['user_id', 'meal_id', 'ratings', 'name', 'meal_type', 'vegan',
       'ovovegan', 'lacvegan', 'lacovovegan', 'pesce'],
      dtype='object')
(7667, 10)
(75, 10)
(68, 10)
(57, 10)
(87, 10)




Index(['user_id', 'meal_id', 'ratings', 'name', 'meal_type', 'vegan',
       'ovovegan', 'lacvegan', 'lacovovegan', 'pesce'],
      dtype='object')
(2114, 10)
(21, 10)
(20, 10)
(16, 10)
(22, 10)






In [298]:
temp.shape

(32082, 8)

In [308]:
break_fast_index = temp[temp['meal_type'] == 'breakfast'].sample(200)['meal_id']

lunch_index = temp[temp['meal_type'] == 'lunch'].sample(200)['meal_id']

dinner_index = temp[temp['meal_type'] == 'dinner'].sample(200)['meal_id']

snack_index = temp[temp['meal_type'] == 'snack'].sample(200)['meal_id']

meals = []

for user in user_table['user_id']:
    for bf in break_fast_index.sample(50):
        meals.append([user,bf])
    for bf in lunch_index.sample(50):
        meals.append([user,bf])
    for bf in dinner_index.sample(50):
        meals.append([user,bf])
    for bf in snack_index.sample(100):
        meals.append([user,bf])

meals_df = pd.DataFrame(meals,columns=['user_id','meal_id'])

meals_df.shape

(750000, 2)

In [309]:
meals_df['ratings'] = np.random.randint(4,11,750000)

In [310]:
meals_df = pd.merge(meals_df, temp, left_on='meal_id', right_on='meal_id', how='inner')

meals_df.columns

meals_df = meals_df[['user_id', 'meal_id', 'ratings','meal_type']]

In [315]:
meals_df = meals_df.sample(200000).sort_index()

In [316]:
csv_buffer = StringIO()
meals_df.to_csv(csv_buffer, header=True, index=False)

In [317]:
t = s3.put_object(Bucket='mymacrochefhj', Key='Resources/Data/ratings.csv', Body=csv_buffer.getvalue())

In [320]:
for d in ['vegan', 'ovovegan', 'lacvegan', 'lacovovegan', 'pesce']:
    selected_meals = pd.merge(newclusterDF, meals_df, left_on='user_id', right_on='user_id', how='inner')
    selected_meals = selected_meals[['user_id','meal_id','ratings']]
    selected_meals = pd.merge(selected_meals, temp, left_on='meal_id', right_on='meal_id', how='inner')
#     print(selected_meals.columns)
    selected_meals = selected_meals[selected_meals[d] == 1]
    print(d)
    print(selected_meals.shape)
    breakfast_meals = selected_meals[selected_meals.meal_type == 'breakfast'].sort_values(by='ratings', ascending=False)

    breakfast_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(breakfast_meals.shape)

    b = list(breakfast_meals.head(28)['meal_id'])
    bf_meals = []
    for i in b:
        bf_meals.append(temp[temp['meal_id']==i]['name'].values[0])
    lunch_meals = selected_meals[selected_meals.meal_type == 'lunch'].sort_values(by='ratings', ascending=False)

    lunch_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(lunch_meals.shape)

    l = list(lunch_meals.head(28)['meal_id'])
    l_meals = []
    for i in l:
        l_meals.append(temp[temp['meal_id']==i]['name'].values[0])

    dinner_meals = selected_meals[selected_meals.meal_type == 'dinner'].sort_values(by='ratings', ascending=False)

    dinner_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(dinner_meals.shape)

    d = list(dinner_meals.head(28)['meal_id'])
    d_meals = []
    for i in d:
        d_meals.append(temp[temp['meal_id']==i]['name'].values[0]) 

    snack_meals = selected_meals[selected_meals.meal_type == 'snack'].sort_values(by='ratings', ascending=False)

    snack_meals.drop_duplicates('meal_id', inplace=True, keep='first')
    print(snack_meals.shape)

    s = list(snack_meals.head(28)['meal_id']) 
    s_meals = []
    for i in s:
        s_meals.append(temp[temp['meal_id']==i]['name'].values[0])
    
#     print(s_meals)
#     print(bf_meals)
#     print(l_meals)
#     print(d_meals)
    
    print('\n\n\n')

vegan
(709, 10)
(27, 10)
(22, 10)
(26, 10)
(19, 10)




ovovegan
(1559, 10)
(57, 10)
(46, 10)
(46, 10)
(49, 10)




lacvegan
(872, 10)
(34, 10)
(24, 10)
(28, 10)
(26, 10)




lacovovegan
(4922, 10)
(163, 10)
(132, 10)
(132, 10)
(165, 10)




pesce
(1161, 10)
(33, 10)
(50, 10)
(41, 10)
(30, 10)






In [322]:
sagemaker.delete_endpoint('kmeans-2019-12-17-23-25-25-039')

AttributeError: module 'sagemaker' has no attribute 'delete_endpoint'